In [4]:
            # Store tools for execution
        self.tools = tools or []
        self.tool_callables = {}
        
        # Register callable tools
        if tools:
            for tool in tools:
                if callable(tool):
                    self.tool_callables[tool.__name__] = tool
    
    def _execute_tool(self, function_name: str, arguments: Dict[str, Any]) -> Any:
        """
        Execute a function from the registered tools.
        
        Args:
            function_name: Name of the function to execute
            arguments: Arguments to pass to the function
            
        Returns:
            Function result or error message
        """
        if function_name not in self.tool_callables:
            error_msg = f"Function '{function_name}' not found in registered tools"
            logger.error(error_msg)
            return {"error": error_msg}
            
        try:
            func = self.tool_callables[function_name]
            logger.info(f"Executing function: {function_name} with arguments: {arguments}")
            result = func(**arguments)
            return result
        except Exception as e:
            error_msg = f"Error executing function '{function_name}': {str(e)}"
            logger.error(error_msg)
            return {"error": error_msg}
    
    def _execute_function_calls(self, function_calls: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Execute a list of function calls.
        
        Args:
            function_calls: List of function calls from model
            
        Returns:
            List of function results
        """
        results = []
        
        for call in function_calls:
            # Extract function name and arguments based on format
            name = call.get("name")
            if not name:
                name = call.get("functionCall", {}).get("name")
                
            # Get arguments, handling different formats
            args = call.get("args", {})
            if not args:
                args_raw = call.get("arguments") or call.get("functionCall", {}).get("arguments")
                if isinstance(args_raw, str):
                    try:
                        args = json.loads(args_raw)
                    except json.JSONDecodeError:
                        args = {}
                else:
                    args = args_raw or {}
            
            if not name:
                logger.warning("Function call missing name")
                continue
                
            # Execute the function
            result = self._execute_tool(name, args)
            
            # Add to results
            results.append({
                "name": name,
                "args": args,
                "result": result
            })
            
        return results